# Import packages

In [ ]:
import sys
sys.path.insert(0, "../src")

from datahandler.datahandler import DataHandler
from segmentation.segmentation import Segmentation
from pointcloud.pointcloud import PointCloudManip
import utils.utils as utils

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

# Set variables

In [ ]:
root = "../data/data_10_31"
img_path = "capture-31-10-1"
calib_path = "calibration_2022_10_31_14_59_14_1667224754485089818"

# Can be None or ["215122254778", "151422251878", "215122256544", "213522251068"]
cams_to_keep = None

# Load data

In [ ]:
dh_ = DataHandler(root, img_path, calib_path, cams_to_keep)

In [ ]:
person_rgb, person_depth = dh_.load_images("mans")
bg_rgb, bg_depth = dh_.load_images("background")

trans = dh_.load_transformations()
depth_scales = dh_.load_depth_scales()
intrinsics = dh_.load_intrinsics()

In [ ]:
print(depth_scales)

In [ ]:
print(trans)

In [ ]:
print(intrinsics)

In [ ]:
utils.plot_rgbd(list(person_rgb.values()), list(person_depth.values()))

In [ ]:
utils.plot_rgbd(list(bg_rgb.values()), list(bg_depth.values()))

# Segment person

In [ ]:
seg = Segmentation()

In [ ]:
out_rgb, out_dep = seg.segment_person(
    list(person_rgb.values()),
    list(person_depth.values())
)

In [ ]:
utils.plot_rgbd(out_rgb, out_dep)

# Depth filtering

In [ ]:
thr = 1700
out_dep_filt = [np.where(im <= thr, im, 0) for im in out_dep]

In [ ]:
utils.plot_rgbd(out_rgb, out_dep_filt)

# Create Point Cloud

In [ ]:
pcd = PointCloudManip.create_point_cloud(
    (
        dict(zip(person_rgb.keys(), out_rgb)),
        dict(zip(person_depth.keys(), out_dep_filt)),
    ),
    depth_scales,
    trans,
    intrinsics
)

In [ ]:
o3d.visualization.draw_geometries([pcd])

### Distance filtering

In [ ]:
pcd_person = PointCloudManip.create_point_cloud(
    (
        person_rgb,
        person_depth,
    ),
    depth_scales,
    trans,
    intrinsics
)

In [ ]:
pcd_bg = PointCloudManip.create_point_cloud(
    (
        bg_rgb,
        bg_depth,
    ),
    depth_scales,
    trans,
    intrinsics
)

In [ ]:
pcd_f = PointCloudManip.distance_filter(pcd_person, pcd_bg, 0.06)


In [ ]:
o3d.visualization.draw_geometries([pcd_f])

### Segment out plane

In [ ]:
pcd_s = PointCloudManip.segment_out_plane(pcd_person, 0.05)

In [ ]:
o3d.visualization.draw_geometries([pcd_s])

### Simple Crop

In [ ]:
print(f"Min bounds: {pcd_s.get_min_bound()}")
print(f"Max bounds: {pcd_s.get_max_bound()}")

In [ ]:
min_bounds = (-1.0, -2.0, 0.0)
max_bounds = (1.0, 2.0, 2.0)
pcd_c = PointCloudManip.crop_pcd(pcd_s, min_bounds, max_bounds)

In [ ]:
o3d.visualization.draw_geometries([pcd_c])

# Create Mesh

In [ ]:
mesh = utils.create_mesh_tsdf(
    dict(zip(person_rgb.keys(), out_rgb)),
    dict(zip(person_depth.keys(), out_dep)),
    depth_scales,
    trans,
    intrinsics)

In [ ]:
o3d.visualization.draw_geometries([mesh])